In [ ]:
# Importamos la librería que necesitamos para el archivo xml
import xml.etree.ElementTree as ET 
import os

In [ ]:
# Creamos una clase para automatizar la limpieza y apertura de los datos
# definimos el constructor
class ListaAutomatica:
    def __init__(self, ruta, nombre_archivo):
        self.ruta = ruta
        self.nombre_archivo = nombre_archivo
    
# Definimos el método de apertura del archivo, que nos devuelve el contenido del archivo en el return
    def apertura(self):
        tree = ET.parse(self.ruta + self.nombre_archivo)
        root = tree.getroot()


        xml_list = []
        for child in root:
            xml_dict = {}
            for subchild in child:
                xml_dict[subchild.tag] = subchild.text
            xml_list.append(xml_dict)
        return xml_list


# Definimos el método de limpieza del archivo, el cual nos devolverá una lista de tuplas en el return
    def limpieza_xml(self, lista):                  
        self.lista = lista 
        lista_tuplas = []
        
        for i in lista: 
            i.pop('level_0')
            if i.get('gender') == '0':
                i['gender'] = 'Man'

            elif i.get('gender') == '1':
                i['gender'] = 'Woman'

            elif i.get('gender') == '2':
                i['gender'] = 'Nonbinary'
            
            elif i.get('gender') == '3':
                i['gender'] = 'Prefer not to say'

            elif i.get('gender') == '4':
                i['gender'] = 'Prefer to self-describe'

        for diccionario in lista:
            xml_automatizado = tuple(diccionario.values())
            lista_tuplas.append(xml_automatizado)
        
        lista_final = []

        for i in lista_tuplas:

            lista_i = list(i)
            lista_i.append(lista_i[0])
            tupla_i = tuple(lista_i)
            lista_final.append(tupla_i)
        
        return lista_final

In [ ]:
archivo_xml = ListaAutomatica('../datos/', 'data_xml.xml')

In [ ]:
archivo_xml.ruta

In [ ]:
lista_xml = archivo_xml.apertura()

In [ ]:
archivo_limpio_xml = archivo_xml.limpieza_xml(lista_xml)

In [ ]:
archivo_limpio_xml

In [ ]:
import mysql.connector

In [ ]:
# Establecemos la conexión con la base de datos en SQL y realizamos la inserción de datos en la tabla xml
cnx = mysql.connector.connect(user='root', password='AlumnaAdalab',
                              host='127.0.0.1', database='da_proyecto_modulo1')


mycursor = cnx.cursor()
sql = """INSERT INTO tabla_xml (
index_xml,
time,
age,
gender,
index_sql
) VALUES (%s,%s,%s,%s,%s)"""
val = archivo_limpio_xml

try: 
    mycursor.executemany(sql, val)
    cnx.commit()
    print(mycursor.rowcount, "registro/s insertado/s.")

except mysql.connector.Error as err:
    print(err)
    print("Error Code:", err.errno)
    print("SQLSTATE", err.sqlstate)
    print("Message", err.msg)